# Multi-Cluster Optimization to reduce costs

## Prerequisites

- Cluster Controller installed to K8s clusters.
- The `cnatix` CLI is installed.
- The user who runs the test plan is an admin of the root org.
- `cnatix login` has already run.

To verify that a user is an admin of the root org, run:

In [ ]:
# Verify that the role binding is set to the <root-org>:admin.
cnatix iam users self

## Test case 1: Measure current utilization and identify excess capacity in K8S clusters & excess capacity identified representative test workload via CLI

### Expected Results

- A user is able to view the current utilization of K8s clusters.
<!-- - A user is able to view the current utilization of test workloads. -->

### Test Execution Steps

Go to https://app.cloudnatix.com and check the aggregated utilization as well as utilization of individual VMs.

Also the "Clusters" page shows the list of registered clusters. If you
click one of the clusters, you can see the max CPU/memory usage of
each node in the cluster.

<!-- TODO(kenji): Describe a way to view the current utilization of workloads (from the Recommendation" tab of the "Workloads" table or from CLI). -->

## Test case 2: View K8s test workload level cost attribution and optimization recommendations (CPU / memory), perform what-if “impact” analysis (CPU / memory)

### Expected Results

- A user is able to view the recommended resource requests of a test workload.

### Test Execution Steps

If you don't have any workload good for this test, here's an example command to launch an nginx workload.

In [ ]:
kubectl apply --namespace default --filename -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.14.2
        ports:
        - containerPort: 80
        resources:
          requests:
            memory: 512Mi
            cpu: 250m
          limits:
            memory: 512Mi
            cpu: 250m

Run:

In [ ]:
CLUSTER_NAME=<cluster name>
KIND=<kind (e.g., Deployment)>
NAMESPACE=<namespace (e.g., default)>
NAME=<name (e.g., nginx)>

cnatix recommendations workloads summary \
  --cluster-name ${CLUSTER_NAME} \
  --kind ${KIND} \
  --namespace ${NAMESPACE} \
  --name ${NAME}

## Test case 3: Implement recommendation using a single CLI command as a one time only operation

### Expected Results

- A user is able to apply the recommendation by specifying the target CPU and memory request.

### Test Execution Steps

Based on the output of the previous test case, set the resource requests. They can be the same value as the
recommended requests or can be an adjusted value.

Run:

In [ ]:
CONTAINER=<container name>
CPU=<CPU request (e.g., 100m)>
MEMORY=<memory request (e.g., 268435456)>

cnatix recommendations workloads apply \
  --cluster-name ${CLUSTER_NAME} \
  --kind ${KIND} \
  --namespace ${NAMESPACE} \
  --name ${NAME} \
  --container ${CONTAINER} \
  --cpu ${CPU} \
  --memory ${MEMORY}

Run the summary command again. Verify that the "APPLIED" is true.

In [ ]:
cnatix recommendations workloads summary \
  --cluster-name ${CLUSTER_NAME} \
  --kind ${KIND} \
  --namespace ${NAMESPACE} \
  --name ${NAME}

## Test case 4: Implement recommendation using a single CLI command as a "Autopilot" operation

### Expected Results

- A user is able to apply the recommendation without specifying the target CPU or memory request.

### Test Execution Steps

Run the `apply` command without specifying `--cpu`, `--memory`, and `--container`.

In [ ]:
cnatix recommendations workloads apply \
  --cluster-name ${CLUSTER_NAME} \
  --kind ${KIND} \
  --namespace ${NAMESPACE} \
  --name ${NAME} \

The recommendations are applied to all containers in the workload.

Run the `summary` command and check the saving made (the "DIFFERENCE" column).

In [ ]:
cnatix recommendations workloads summary \
  --cluster-name ${CLUSTER_NAME} \
  --kind ${KIND} \
  --namespace ${NAMESPACE} \
  --name ${NAME}

Please note that resource requests are updated only if the difference
between the current requests and recommended values excess a certain
threshold.

## Test case 5: Measure resulting CPU cores / memory GB reclaimed as a % of test workload over 1 week using Autopilot

Run the summary command again after one week passes.

## Test case 6: Measure cluster-level recommendations

### Expected Results

- A user is able to check the cluster-level recommendations. A recommendation provides the new
  instance types with the number of nodes and the possible cost improvements.

### Test Execution Steps

After one week from the cluster set up, run:

In [1]:
cnatix recommendations clusters list \
  --cluster-name ${CLUSTER_NAME} \
  --constraints "NO_CONSTRAINT" \
  --resources-sources "REQUEST"

cnatix: command not found


: 127

It shows the summary of cluster-level recommendations (new instance types with new numbers of
nodes) per day with the new expected costs.

Next, run the following:

In [ ]:
cnatix recommendations clusters list \
  --cluster-name ${CLUSTER_NAME} \
  --constraints "NO_CONSTRAINT" \
  --resources-sources "REQUEST" \
  --days 1

This shows the list of individual recommendations in a day with the UUID. Pick up one of
the recommendations, copy its UUID, and run the following to see the details of a recommendation:

In [ ]:
cnatix recommendations clusters get --uuid ${UUID}

For the list commands, you can see different results by changing the values of
`--constraints` flag and `--resources-sources` flag.  The valid values are:

- `constraints=SAME_INSTANCE_TYPES`: do not change the instance types, just change the number
  of nodes.
- `constraints=SAME_INSTANCE_FAMILY`: limit the instance types within the ones in the currently
  used instance family.
- `resources-sources=VPA_RECOMMENDED`: use the recommended resource requirements rather than
  specified ones. Wouldn't change the results if all of the workloads are already in autopilot mode.
- `resources-sources=VPA_LOWER_BOUND`: use the lower-bound resource requirements.
